In [ ]:
!pip install cshogi
!pip install dlshogi==0.1.1

In [ ]:
#!git clone https://github.com/TadaoYamaoka/python-dlshogi2.git

In [ ]:
#%cd python-dlshogi2
#!pip install -e .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
# from pydlshogi2.dataloader import HcpeDataLoader

In [ ]:
device = torch.device("cpu")

In [ ]:
from dlshogi.data_loader import DataLoader

In [ ]:
train_dataloader = DataLoader(DataLoader.load_files(["/content/drive/My Drive/ShogiAIBookData/dlshogi_with_gct-001.hcpe"]), 32, device, shuffle=False)

In [ ]:
from dlshogi.common import *
from dlshogi.network.policy_value_network import policy_value_network
from dlshogi import serializers
from dlshogi import cppshogi

In [ ]:
model = policy_value_network("resnet10_swish", add_sigmoid=True)
model.set_swish(False)#swishをx*sigmoid(x)で計算するモード
serializers.load_npz("/content/drive/My Drive/ShogiAIBook/model/model_resnet10_swish-072", model, False)

In [ ]:
model.eval()

PolicyValueNetworkAddSigmoid(
  (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l10): Conv2d(192, 192, k

In [ ]:
x1s = []
x2s = []
moves = []
results = []
n_batch = 32
for x1, x2, move, result, value in train_dataloader:
  # x1 = [32, 62, 9, 9], x2 = [32, 57, 9, 9]
  pred_move, pred_result = model(x1, x2)
  # pred_mode: [32, 2187], pred_result: [32, 1]
  x1s.append(x1.detach().numpy())
  x2s.append(x2.detach().numpy())
  moves.append(pred_move.detach().numpy())
  results.append(pred_result.detach().numpy())
  if len(x1s) >= n_batch:
    break

In [ ]:
with open("/content/drive/My Drive/ShogiAIBook/model/SampleIO.bin", "wb") as f:
  for arys in [x1s, x2s, moves, results]:
    d = np.concatenate(arys, axis=0).tobytes()
    f.write(d)